In [ ]:
# Instalação e importação dos pacotes necessários para extração e requisição dos dados, classificação das etnias
# e manipulação da tabela
# Installation and importantion of necessary packages for extraction and retrieval, race classification and table manipulation

!pip3 install opencv-python deepface numpy selenium
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import cv2
import numpy as np
import  time
import cv2 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 

In [ ]:
# Variável que evita o bloqueio do código pelo site
# Variable that prevents the code from being blocket by the website

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [ ]:
# Essas listas serão o resultado da extração. Você pode adicionar novas listas incluindo a quantidade de partidas etc.
# These lists will be the result of the extraction. You can add new lists, such as the number of matches etc.
Lista_de_Jogadores = []
Gols_Marcados = []
Fotos = []

# Criação de uma função que fará a extração dos dados
# Creation of a function that will perform dates extraction

def extraçao():
    page = input("Insira a página que terá os dados extraídos:\n")
    pageTree = requests.get(page, headers=headers)
    
# Na variável "driver", é necessário indicar o caminho no diretório do seu computador onde está o driver utilizado
# In the "driver" variable, it will be necessary to indicate the patch in yout computer's directory where the driver is located

    driver = webdriver.Chrome(r'C:\Users\arthu\Downloads\chromedriver_win32 (1)\chromedriver.exe')
    
# A passagem no selenium é feita a partir da localização do elemento com o título "Para a página seguinte"
# The "click" in selenium was made from localization of element "Para a página seguinte"
    driver.implicitly_wait(30)
    driver.get(page)
    
# O código de extração
# The extraction code
    while True:
        try:
            next_page_element = WebDriverWait(driver, 10).until(
                
# A passagem no selenium é feita a partir da localização do elemento com o título "Para a página seguinte"
# The "click" in selenium was made from localization of element "Para a página seguinte"
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[title='Para a página seguinte']")))
            page_html = driver.page_source     
            pageSoup = BeautifulSoup(page_html, "html.parser")
            
# A partir da análise do HTML feita pelo BeautfulSoup, ele irá raspar os dados que informamos.
# With the HTML analysis complete for BeautfulSoup, we can start extract the informations.
# 1) O nome dos jogadores, tendo como elemento âncora a classe hauptlink dentro da tabela principal da página
# 1) The players, using like ancor element the hauptlink class inside a major table in the page.
            Jogadores = []
            nomes = pageSoup.find_all(lambda tag: tag.name == "td" and tag.get("class") == ["hauptlink"])
            for i in range (0, 25):
                Jogadores.append(nomes[i].text)
            Nome_do_Jogador = [jogador.strip() for jogador in Jogadores]

# 2) A quantidade de gols marcadas, tendo como elemento âncora a classe zentriert hauptlink dentro da tabela principal da página
# 2) The second information is the goals scored. For this, I used the zentriert hauptlink like ancor element.

            Quantidade_de_Gols=[]
            Gols = pageSoup.find_all("td", {"class": "zentriert hauptlink"})
            for i in range (0, 25):
                Quantidade_de_Gols.append(Gols[i].text)
                
# 3) As imagens dos jogadores que serão tratadas posteriormente pelo Deepface. Aqui, importamos o url onde estão essas imagens.
# Note que ao invés de definimos uma classe em HTML, inserimos a infomração "alt: [Nome_do_Jogador]". Isso porque a classe
# dessas URLs era a mesma da classe dos jogadores, o que impedia a extração. Por isso colocamos o título dessas URLs como 
# parâmetro da extração
# Em uma temporada específica, o código travava em duas imagens sem data-src, o que gerava um problema que não consegui resolver,
#aparentemente um bug. Por isso inclui uma condicional para, caso não fosse encontrado o data-src, fosse inserido um valor específico
# 3) The images that will be used later by Deepface. Here, we import the URLs where these images are located.
# Instead of defining an HTML class, we inserted the information "alt:[player_list]". This is because the class of these URLS
# was the same as the "players names", wich prevented the extraction. Therefore, we used the titles of these URLs as extraction
# parameters.
# In a specific season, the code would get stuck on two images without a data-src atribute, causing an issue that we couldn't
# resolve, possibly a bug. As a solution, I included a conditional statement to check if the "data-src" was not found, and in 
# that case, a specific value would be inserted
            valor_padrao = "Link não encontrado"
            imagens_jogadores = pageSoup.find_all("img", {"alt": [Nome_do_Jogador]})
            urls_imagens = [img.get("data-src") or valor_padrao for img in imagens_jogadores]
            for url_imagem in urls_imagens:
                Foto = urls_imagens

# Após as extraçãoes, as informações são adicionadas às listas criadas antes do código de raspagem
# After the extractions, the information will be added to the lists created before the scraping code
            Lista_de_Jogadores.extend(Nome_do_Jogador)
            Fotos.extend(urls_imagens)
            Gols_Marcados.extend(Quantidade_de_Gols)

# Ao final do processo, o Selenium espera 5 segundos e busca o "elemento" necessário para realizar um click e passar para a 
# próxima página
# At the end of the process, Selenium waits 5 seconds and searchs for the necessary "element" to perform a click and move to 
# the next page
            next_page_element.click()
            time.sleep(5)
            page_html = driver.page_source

# Aqui ele faz uma nova leitura da nova página e retorna o loop, caso verdadeiro
# Here, the code performs a new read of the page and returns to the loop if it is true
            pageSoup = BeautifulSoup(page_html, "html.parser")

# Caso seja falso, ele entra na exceção causada pelo TimeoutException
# If it is false, it enters the exception caused by TimeoutException
        except TimeoutException as e:

# Para impedir que uma mensagem de erro seja mostrada na tela, inserimos uma mensagem personalizada seguida do break
# To prevent an error message from being displayer, I inserted a customized message followed by the break command
            e = "Busca finalizada"
            print(e)
            break

# Após o break, o sistema faz uma extração na última página. Nessa parte foi feita uma pequena alteração no código. Como não 
# sabemos quantos jogadores aparecem na última página, limitamos a quantidade de extração dos "Jogadores" pelo tamanho da lista
# de Quantidades de Gols.
# After the break, the system performs an extraction on the last page. In this part, a small modification was made to the code.
# Since we don't know how many players appear on the last page, we limited the extraction of "Players" based on the size of 
# the "Goals Quantity" list
    page_html = driver.page_source
    pageSoup = BeautifulSoup(page_html, "html.parser")

    Quantidade_de_Gols = []
    Gols = pageSoup.find_all("td", {"class": "zentriert hauptlink"})
    for Gol in Gols:
        Quantidade_de_Gols.append(Gol.text)

    Jogadores = []
    nomes = pageSoup.find_all(lambda tag: tag.name == "td" and tag.get("class") == ["hauptlink"])
    for i in range(0, len(Quantidade_de_Gols)):
        Jogadores.append(nomes[i].text)
    Nome_do_Jogador = [jogador.strip() for jogador in Jogadores]
    
    valor_padrao = "Link não encontrado"
    imagens_jogadores = pageSoup.find_all("img", {"alt": [Nome_do_Jogador]})
    urls_imagens = [img.get("data-src") or valor_padrao for img in imagens_jogadores]

    Lista_de_Jogadores.extend(Nome_do_Jogador)
    Gols_Marcados.extend(Quantidade_de_Gols)
    Fotos.extend(urls_imagens)

# Exibe uma mensagem de conclusão
# Show a finish message
    print("Extração adicional concluída")
    driver.quit()

# E oferece ao usuário a opçao de fazer uma nova extração de dados, realizando novamente o loop anterior caso a resposta seja
# positiva
# Offer a user the option for perform a new extraction.
    mais_dados = input("Deseja extrair os dados de outra página? Responda sim ou não.\n")
    resposta = mais_dados.lower()
    if resposta == "sim":
        print("OK")
    else:
        print("Ok, agora vamos montar a tabela com os resultados extraídos")
extraçao ()



In [ ]:
# Nessa parte retirei as importações defeituosas dos links das imagens
# In this part, we removed the faulty imports of the the urls_images

Fotos = [foto for foto in Fotos if foto != "Link não encontrado"]

In [ ]:
# Após a extração dos dados, as listas são incluídas em uma tabela inicial e mesclada a partir da coluna Jogadores
# Aqui, somamos os goals e unimos as URLs das fotos
# After the data extraction, the lists are inserted into an initial table and merged based on the "Jogadores" colunm using an 
# anchor element. Here, we sum the goals and combine the photo URLs
Gols_marcados = [int(valor) for valor in Gols_Marcados]

df = pd.DataFrame({"Jogadores": Lista_de_Jogadores, "Gols": Gols_marcados, "Links das Fotos": Fotos})
df_agrupado = df.groupby('Jogadores')['Gols'].sum().reset_index()
df_unido = df.groupby('Jogadores')['Links das Fotos'].apply(lambda x: ', '.join(x)).reset_index()
resultado = pd.merge(df_agrupado, df_unido, on='Jogadores').sort_values(by="Gols", ascending = False)

df_tabela = pd.DataFrame({
    'Jogadores': resultado['Jogadores'],
    'Gols': resultado['Gols'],
    'Links das Fotos': resultado['Links das Fotos']
})

# Para não haver mais de uma URL para cada jogador, excluimos todos os elementos após a ',' de cada linha dos Links das Fotos
# To avoid having multiple URLs for each player, we removed all elements after the comma (',') in each row of the photo links.
df_tabela['Links das Fotos'] = df_tabela['Links das Fotos'].apply(lambda x: x.split(',')[0])


# Exibir o DataFrame como tabela
# Show a example Dataframe
print(df_tabela.to_string)

# Exporting the DataFrame as a temporary table in Excel

df_tabela.to_excel('Jogadores e etnias entre os artilheiros da La Liga.xlsx', index=False)


In [ ]:
# Apos estarmos com a tabela criada, é hora de iniciarmos a utilização do Deepface para classificarmos, a partir da IA, a etnia 
# dos jogadores
# Once we have the table created, it's time to start using Deepface to classify the players' ethnicity based on AI.


lista_de_fotos = []
dataframe = pd.read_excel('Jogadores e etnias entre os artilheiros da La Liga.xlsx')
coluna = dataframe['Links das Fotos']


In [ ]:
# Importação do módulo necessário do DeepFace para a análise 
# Importing the necessary module from DeepFace for analysis.

from deepface import DeepFace

# Com a lista anteriormente criada, o DeepFace e o Numpy vão abrir os links individualmente, entregar para a IA, fazer a análise
# e retornar uma nova lista com os resultados
# With the previously created list, DeepFace and Numpy will open the links individually, provide them to the AI for analysis, 
# and return a new list with the results.

lista_etnias = []

# Inseri essa contagem para que possamos acompanhar o progresso da análise, já que demora um pouco
# I added this counter so that we can track the progress of the analysis, as it takes some time.
contagem = 0

for image_url in coluna:
    try:
        etnias = []
        response = requests.get (image_url)
        image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        temp_filename = "imagem.jpg"
        cv2.imwrite(temp_filename, image)
        analise = DeepFace.analyze(img_path=temp_filename, actions = ('race',))
        etnias = analise[0]['dominant_race']
        lista_etnias.append(etnias)
        contagem +=1
        print(contagem)
    except:

# Alguns jogadores não tem foto ou a IA não consegue, a partir de seus cálculos, definir um elemento preponderante entre as 
# etnias. Quando isso ocorrer, o programa deve inserir a classificação informada ao invés do erro.
# Some players do not have a photo or the AI cannot determine a predominant ethnicity based on its calculations. When this 
# happens, the program should insert the provided classification instead of the error.

        etnias = "Não detectada"
        lista_etnias.append(etnias)
        contagem +=1
        print(contagem)
    


In [ ]:
# Ao finalizar a análise, o macro pega os dados gerado pela análise e adiciona na tabela em sua posição correspondente
# Upon completion of the analysis, the macro takes the data generated by the analysis and adds it to the table in their 
# corresponding positions.

dataframe = pd.read_excel('Jogadores e etnias entre os artilheiros da La Liga.xlsx')
dataframe['Etnias'] = lista_etnias
dataframe.to_excel('Jogadores e etnias entre os artilheiros da La Liga.xlsx', index=False)